In [ ]:
# !pip install datasets
# !pip install transformers
# !pip install rouge_score
# !pip install sentencepiece

# Hyperparameter Tuning for 'Unsummarization'

### In this notebook, we are exploring how hyperparameters change the output generated by the finetuned 'unsummarization' model. 

### These parameters include -- num_beams, temperature, min_length, repetition penalty, and length penalty

In [ ]:
import pandas as pd

In [ ]:
import torch

In [ ]:
from transformers import T5ForConditionalGeneration, AutoModelForSeq2SeqLM

custom_model = "caffsean/t5-large-finetune-keyword-to-text-generation"

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained('t5-large')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(custom_model)

In [ ]:
from tqdm import tqdm
import time

In [ ]:
def generate_samples_from_word(word_list, beams_list, length_penalty_list, repetition_penalty_list, min_length_list, temperature_list):
    generated_text = []
    for word in tqdm(word_list):
      for num_beams in tqdm(beams_list):
        for repetition_penalty in repetition_penalty_list:
          for length_penalty in length_penalty_list:
            for min_length in min_length_list:
              for temperature in temperature_list:
                start = time.time()

                encoded_input = tokenizer("summarize: " + word)
                with torch.no_grad():
                      generated_ids = model.generate(
                            input_ids = torch.LongTensor(encoded_input['input_ids']).unsqueeze(0),
                            attention_mask = torch.LongTensor(encoded_input['attention_mask']).unsqueeze(0), 
                            min_length=min_length,
                            max_length=60,
                            num_beams=num_beams,
                            temperature=temperature,
                            repetition_penalty=float(repetition_penalty), 
                            length_penalty=float(length_penalty), 
                            early_stopping=True
                            )
                text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
                print(f'beams: {num_beams} - lenghth_pen: {length_penalty} - repetition_pen: {repetition_penalty} - min_length: {min_length}- OUTPUT: {text}')
                
                stop = time.time()
                duration = stop-start
                
                generated_text.append([word,num_beams,length_penalty,repetition_penalty,min_length,temperature,duration,text[0]])
        
    return pd.DataFrame(generated_text, columns=['keyword','num_beams','length_penalty','repetition_penalty','min_length','temperature','duration','output_text'])

In [ ]:
word_list = ['moose','robotic','candle','capture','tongue']
beams_list = [2,4,8,16,24,32]
length_penalty_list = [5]
repetition_penalty_list = [10,20,40]
min_length_list = [50]
temperature_list = [1.0]


sample_grid = generate_samples_from_word(word_list,beams_list,length_penalty_list,repetition_penalty_list,min_length_list, temperature_list)

### After some initial tests it became clear that the length penalty did not have much of an effect and the repetition penalty only changed the results for longer min_lengths. As such, the length penalty was arbitrarily set to 5 and the min_length was set to 50. Moreover, temperature does not appear to impact the output with this model. Most of the variation seems due to changes in keyword and repetition penalty. 

In [ ]:
#sample_grid.to_csv('/content/drive/MyDrive/NLP_stuff/sample_grid.csv')

In [ ]:
for x in sample_grid_df[(sample_grid_df['num_beams'] == 32) & (sample_grid_df['repetition_penalty'] == 40)]['output_text']:
  print(f"{x} \n") 

There were moose in the woods, but there was no evidence that they had been hunted or taken into captivity for any purpose other than to be eaten by their prey." "I don't see why you shouldn't try it," said I. 

He had a robotic smile on his face, but it was not the kind of smile that made him want to say anything more than “I’ve got something for you.” His eyes were reddened when they met each other's faces. 

He had a candle in his hand, and as soon as the sun went down it was too dark for him to see or hear anything; so I told him not to look at any more light than that which came out of my lamp-stand. 

At the time of his capture, however, he was held in captivity and sentenced to three years’ hard labor by an army which had not yet arrived at its final resting-place; but that did not prevent him from being executed. 

It was a tongue-twisting sound, and I could hear it in my mind’s ears; but the noise had to be muffled or drowned out by some kind of loud music which must have come

In [ ]:
sample_grid_df = pd.read_csv('/content/drive/MyDrive/NLP_stuff/sample_grid.csv')
for idx in range(len(sample_grid_df)):
  print(f"{sample_grid_df.iloc[idx]['keyword']}: [Beams: {sample_grid_df.iloc[idx]['num_beams']} - Penalty:{sample_grid_df.iloc[idx]['repetition_penalty']}] - \n\n RESULT: {sample_grid_df.iloc[idx]['output_text']} \n")


moose: [Beams: 2 - Penalty:10] - 

 RESULT: The moose had been killed, and I was told that it would not be long before the next one appeared.’ ‘I have no time to think of anything but this!’ said Mr. Smith. “You are right.” And so we went on. 

moose: [Beams: 2 - Penalty:20] - 

 RESULT: The moose had been killed, and I was told that it would not be long before the next one appeared.’ ‘I have no time to think of anything else but this,” said Mr. Smith. “You know what?” asked Mrs. Jones. 

moose: [Beams: 2 - Penalty:40] - 

 RESULT: The moose had been killed, and I was told that it would not be long before the next one appeared.’ ‘I have no time to think of anything else but this,” said Mr. Smitherman with an air of resignation. 

moose: [Beams: 4 - Penalty:10] - 

 RESULT: The moose, however, was not to be seen for some time after the fire had been put out; and there were no signs of any danger from its presence in that part of New York. I did not know what to do with it. 

moose: [Bea

### In looking at the results, it's clear that a greater number of 'beams' increases the coherence of the text to the detriment of computational speed. As such, it might be reasonable to cap num_beams at 16. Meanwhile, the repetition_penalty does have an impact on the output, though not as extreme as num_beams. As such, it might be reasonable to maintain the num_beams at 16 and the repetition_penalty at 20. This seems to provide an adequate balance between coherence and computational resources. 

## Let's look at the output generated from a list of random words: 

In [ ]:
import random

df = pd.read_csv('/content/drive/MyDrive/NLP_stuff/keyword_to_text.csv')

word_list = random.choices(list(df['keyword']),k=20)

for word in word_list:
  print(word)

thereafter
dilemma
posted
intermission
cultivate
miniature
throwing
perfectly
bade
bearing
finishing
tightly
violence
agitation
merit
sequestered
headwaters
concerning
adviser
valued


In [ ]:
word_list = word_list
beams_list = [16]
length_penalty_list = [5]
repetition_penalty_list = [20]
min_length_list = [50]
temperature_list = [1.0]


sample_grid_2 = generate_samples_from_word(word_list,beams_list,length_penalty_list,repetition_penalty_list,min_length_list, temperature_list)

In [ ]:
#sample_grid_2.to_csv('/content/drive/MyDrive/NLP_stuff/sample_grid_2.csv')

In [ ]:
for idx in range(len(sample_grid_2)):
  print(f"{sample_grid_2.iloc[idx]['keyword']}: \n RESULT: {sample_grid_2.iloc[idx]['output_text']} \n")


thereafter: 
 RESULT: It was not long thereafter, however, before he found himself in the company of his brother-in-law’s wife and two young sons with whom he had been engaged on an expedition to Asia Minor for some time after their marriage. 

dilemma: 
 RESULT: He was in a dilemma of his own, and had no choice but to look for the solution that would be best for him--the one which could solve all problems at once." "I don't know how you do it," she said. 

posted: 
 RESULT: There was a sign posted on the door that said, “Come and see us.” Then we went in search of some one who could show us how to get there by means of our wagons or horse-drawn carriages. 

intermission: 
 RESULT: Then it was time for intermission, and we sat down to have our last meal of the day before going out into the woods in search of wild boar or some other mammal that might attack us.’ ‘Are you sure?’ she asked. 

cultivate: 
 RESULT: It is necessary, therefore, to cultivate a spirit of self-restraint on the p

## Conclusion:

In looking at the output of the generated sentences, this approach appears to be a reasonable means through which to generate text from single keywords. Not only are the sentences *mostly* well-structured, the keywords are used in an appropriate context and would therefore be suitable for use in a didactic context. Namely, these sentences will be used to generate fill-in-the-blank questions in which the keyword is masked and the student/user would be required to choose from a list of potential answers that includes the original keyword. Of course, more filters would need to be utilized in order to remove offensive or incorrect content from a final repository of public-facing multiple choice questions. 